## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification German

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: `de.deid.clinical`
- **Model Description**: This pipeline can be used to deidentify PHI information from German medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate PATIENT, HOSPITAL, DATE, ORGANIZATION, CITY, STREET, USERNAME, PROFESSION, PHONE, COUNTRY, DOCTOR, AGE, CONTACT, ID, PHONE, ZIP, ACCOUNT, SSN, DLN, PLATE entities.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "de-deid-clinical"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"



### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'


    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
    
    input_data = json.dumps(data, indent=4, ensure_ascii=False) if content_type == 'application/json' else data


    if content_type == 'application/json':
        with open(input_file_name, "w", encoding='utf-8') as f:
            f.write(input_data)
    else:
        with open(input_file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(input_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=data)

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4, ensure_ascii=False)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    with open(output_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_output_dir}/{os.path.basename(output_file_name)}", Body=data)

    return response_data

### Initial Setup

In [7]:
docs = [
    '''Dr. Hans-Wolfgang Weihmann - RM57, Städt Klinikum Dresden-Friedrichstadt, Friedrichstraße 41, Dresden''',

    '''Er arbeitete bis 24.08.1940 - Gärtner bei Planten un Blomen in Hamburg, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. Klein im September.''',
    ]


sample_text = """Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhaus ei... 
Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhaus eingeliefert. Herr Michael Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen.Persönliche Daten :ID-Nummer: T0110053F Platte A-BC124 Kontonummer: DE89370400440532013000 SSN : 13110587M565 Lizenznummer: B072RRE2I55 Adresse : St.Johann-Straße 13 19300 """

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    Example: "**Dr. Hans-Wolfgang Weihmann - RM57, Städt Klinikum Dresden-Friedrichstadt, Friedrichstraße 41, Dresden**"

    - **masked**: Default policy that masks entities with their type.

      -> 'Dr.  `<PATIENT>` - `<USERNAME>`, `<HOSPITAL>`, `<STREET>`, `<CITY>`'

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      -> 'Dr.  `Karl-August Blümel` - `RP400`, `University Hospital Cologne`, `Fadime-Pölitz-Allee`, `Böblingen`'

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

      -> 'Dr.  `****` - `****`, `****`, `****`, `****`'

    - **masked_with_chars**: Masks entities with asterisks (\*).

      -> 'Dr.  [`********************`] - [`**`], [`***********************************`], [`****************`], [`*****`]'

- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').

You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}


### JSON


1. **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

2. **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

#### Example 1: masked (default-policy)

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,Zusammenfassung : <PATIENT> wird am Morgen des <DATE> ins <HOSPITAL>. .. Zusammenfassung : <PATIENT> wird am Morgen des <DATE> ins <HOSPITAL> eingeliefert. Herr <PATIENT> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen. Persönliche Daten :ID-Nummer: <ID> Platte <PLATE> Kontonummer: <ACCOUNT> SSN : <SSN> Lizenznummer: <DLN> Adresse : <STREET> <ZIP>


#### Example 2: obfuscated

In [9]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,Zusammenfassung : Hansgeorg Burger wird am Morgen des 12 Dezember 2018 ins Krankenhaus Reinbek St. Adolf. .. Zusammenfassung : Hansgeorg Burger wird am Morgen des 12 Dezember 2018 ins Krankenhaus am Zoo Karlsruhe eingeliefert. Herr Hansgeorg Burger ist 10 Jahre alt und hat zu viel Wasser in den Beinen. Persönliche Daten :ID-Nummer: Z6109604V Platte W-UJ811 Kontonummer: 192837465738 SSN : 91478295A213 Lizenznummer: Y865HQI6N62 Adresse : Kösterring 4/1 95284


#### Example 3: masked (default-policy)

In [10]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Dr. <DOCTOR> - <USERNAME>, <HOSPITAL>, <STREET>, <CITY>"
1,"Er arbeitete bis <DATE> - <PROFESSION> bei <ORGANIZATION> in <CITY>, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. <DOCTOR> im <DATE>."


#### Example 4: obfuscated

In [11]:
input_json_data = {"text": docs, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Dr. Karl-Dieter Lukas - Jupiter.Schürmann, Klinikum Hannover-Süd, Heinweg 2/9, Hannover"
1,"Er arbeitete bis 24.08.1940 - IT-Administrator bei Belkin in Teterow, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. Sorgatz Junker im September."


### JSON Lines

1. **Input format**: Provide input in JSON Lines format, where each line is a JSON object representing a text document along with any optional parameters.

```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [12]:
import json

def create_jsonl(records, masking_policy):
    json_records = []

    for text in records:
        record = {
            "text": text,
            "masking_policy": masking_policy
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

#### Example 1: masked (default-policy)

In [13]:
input_jsonl_data = create_jsonl(docs, masking_policy="masked")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "Dr. <DOCTOR> - <USERNAME>, <HOSPITAL>, <STREET>, <CITY>"}
{"predictions": "Er arbeitete bis <DATE> - <PROFESSION> bei <ORGANIZATION> in <CITY>, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. <DOCTOR> im <DATE>."}


#### Example 2: obfuscated

In [14]:
input_jsonl_data = create_jsonl(docs, masking_policy="obfuscated")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "Dr. Karl-Dieter Lukas - Jupiter.Schürmann, Klinikum Hannover-Süd, Heinweg 2/9, Hannover"}
{"predictions": "Er arbeitete bis 24.08.1940 - IT-Administrator bei Belkin in Teterow, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. Sorgatz Junker im September."}


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [16]:
import json
import os


input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)


validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"


def write_and_upload_to_s3(input_data, file_name):

    file_format = os.path.splitext(file_name)[1].lower()
    s3_key = f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}"

    if file_format == ".json":
        input_data_json = json.dumps(input_data, indent=4, ensure_ascii=False)

        with open(file_name, "w", encoding='utf-8') as f:
            f.write(input_data_json)
    elif file_format == ".jsonl":
        with open(file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=s3_key, Body=data)

    return input_data


input_json_data = {
    "input1.json": {"text": docs, "masking_policy": "masked"},
    "input2.json": {"text": docs, "masking_policy": "obfuscated"},
    "input3.json": {"text": docs, "masking_policy": "masked_fixed_length_chars"},
    "input4.json": {"text": docs, "masking_policy": "masked_with_chars"}
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}


for file_name, data in input_json_data.items():
    write_and_upload_to_s3(data, f"{json_input_dir}/{file_name}")

for file_name, data in input_jsonl_data.items():
    write_and_upload_to_s3(data, f"{jsonl_input_dir}/{file_name}")



### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [18]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):
    
    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

In [19]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_json_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked


,predictions
0,"Dr. <DOCTOR> - <USERNAME>, <HOSPITAL>, <STREET>, <CITY>"
1,"Er arbeitete bis <DATE> - <PROFESSION> bei <ORGANIZATION> in <CITY>, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. <DOCTOR> im <DATE>."




Masking policy: obfuscated


,predictions
0,"Dr. Karl-Dieter Lukas - Jupiter.Schürmann, Klinikum Hannover-Süd, Heinweg 2/9, Hannover"
1,"Er arbeitete bis 24.08.1940 - IT-Administrator bei Belkin in Teterow, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. Sorgatz Junker im September."




Masking policy: masked_fixed_length_chars


,predictions
0,"Dr. **** - ****, ****, ****, ****"
1,"Er arbeitete bis **** - **** bei **** in ****, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. **** im ****."




Masking policy: masked_with_chars


,predictions
0,"Dr. [********************] - [**], [***********************************], [****************], [*****]"
1,"Er arbeitete bis [********] - [*****] bei [***************] in [*****], verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. [***] im [*******]."


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [21]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [22]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_jsonlines_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked
{"predictions": "Dr. <DOCTOR> - <USERNAME>, <HOSPITAL>, <STREET>, <CITY>"}
{"predictions": "Er arbeitete bis <DATE> - <PROFESSION> bei <ORGANIZATION> in <CITY>, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. <DOCTOR> im <DATE>."}


Masking policy: obfuscated
{"predictions": "Dr. Karl-Dieter Lukas - Jupiter.Schürmann, Klinikum Hannover-Süd, Heinweg 2/9, Hannover"}
{"predictions": "Er arbeitete bis 24.08.1940 - IT-Administrator bei Belkin in Teterow, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. Sorgatz Junker im September."}


Masking policy: masked_fixed_length_chars
{"predictions": "Dr. **** - ****, ****, ****, ****"}
{"predictions": "Er arbeitete bis **** - **** bei **** in ****, verbrannte sich an beiden Beinen - entwickelte Geschwüre. Der Patient konsultierte Dr. **** im ****."}


Masking policy: masked_with_chars
{"predictions": "Dr. [********************] - [**], [*************

In [23]:
model.delete_model()

INFO:sagemaker:Deleting model with name: de-deid-clinical-2024-05-21-06-51-27-974


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

